Importar data

In [1]:
import pandas as pd
noSolutionDF = pd.read_csv('nosolcomas.csv')
solutionDF = pd.read_csv('solcomas.csv')

Etiquetas

In [2]:
import numpy as np
noSolutionCount = noSolutionDF.shape[0]
solutionCount = solutionDF.shape[0]

print(noSolutionCount)
print(solutionCount)

noSolutionLabel = [0] * noSolutionCount * 6 
solutionLabel = [1] * solutionCount * 6
labels = noSolutionLabel + solutionLabel


Y = np.array(labels)

1340
1499


X tableros

In [4]:
def Rotate_90(tablero):
    return np.rot90(tablero, k=1)

def Rotate_180(tablero):
    return np.rot90(tablero, k=2)

def Rotate_270(tablero):
    return np.rot90(tablero, k=3)

def FlipX(tablero):
    return np.fliplr(tablero)

def FlipY(tablero):
    return np.flipud(tablero)


boards = []

for indice, fila in noSolutionDF.iterrows():
    tableros_en_celda = fila['Board'].replace("\r", "").split('\n')
    tableros_en_celda = list(filter(lambda x: x != '', tableros_en_celda))
    tablero_matriz = [list(fila) for fila in tableros_en_celda]
    boards.append(tablero_matriz)
  
    
print("El tamaño de la lista `boards` es:", len(boards))
for indice, fila in solutionDF.iterrows():
    tableros_en_celda = fila['Board'].replace("\r", "").split('\n')
    tableros_en_celda = list(filter(lambda x: x != '', tableros_en_celda))
    tablero_matriz = [list(fila) for fila in tableros_en_celda]
    boards.append(tablero_matriz)

# Data augmentation
aug_boards = []
for board in boards:
    board_np = np.array(board)
    aug_boards.append(board_np)  
    aug_boards.append(Rotate_90(board_np))  
    aug_boards.append(Rotate_180(board_np)) 
    aug_boards.append(Rotate_270(board_np))  
    aug_boards.append(FlipX(board_np))  
    aug_boards.append(FlipY(board_np))  

print("El tamaño de la lista `boards` es:", len(aug_boards))
with np.printoptions(threshold=np.inf):
    print(aug_boards[0])

# Imprimir las listas de tableros
'''for i, lista in enumerate(boards):
    print(f"Lista de tableros {i+1}:")
    for tablero in lista:
        print(tablero)
    print()  # Salto de línea entre listas de tablero'''

El tamaño de la lista `boards` es: 1340
El tamaño de la lista `boards` es: 17034
[[' ' ' ' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#']
 [' ' ' ' '#' ' ' ' ' ' ' '#' '$' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '#']
 [' ' ' ' '#' ' ' '$' ' ' '$' ' ' ' ' ' ' '$' ' ' '#' '$' ' ' '#']
 [' ' ' ' '#' ' ' '#' '$' '#' ' ' '#' '#' '#' ' ' ' ' '$' ' ' '#']
 [' ' ' ' '#' ' ' '#' ' ' '.' '.' '.' '.' '.' '#' '#' '$' '#' '#']
 [' ' ' ' '#' ' ' ' ' '.' '.' '.' '+' '.' '.' ' ' ' ' '$' ' ' '#']
 ['#' '#' '#' '#' '$' '#' ' ' '.' '.' '.' '.' '#' ' ' '#' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' '#' '#' '#' '#' ' ' '#' '$' '$' '$' ' ' '#']
 ['#' ' ' '$' '#' ' ' ' ' ' ' ' ' ' ' '$' ' ' '$' ' ' '#' ' ' '#']
 ['#' ' ' '$' ' ' '.' '.' '#' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#']]


'for i, lista in enumerate(boards):\n    print(f"Lista de tableros {i+1}:")\n    for tablero in lista:\n        print(tablero)\n    print()  # Salto de línea entre listas de tablero'

Tableros a tensor

In [5]:
import numpy as np

def tablero_a_tensor(tablero):
    alto = len(tablero)
    ancho = len(tablero[0])
    tensor = np.zeros((alto, ancho, 7))

    for i in range(alto):
        for j in range(ancho):
            elemento = tablero[i][j]
            if elemento == '#':
                tensor[i, j, 0] = 1
            elif elemento == '$':
                tensor[i, j, 1] = 1
            elif elemento == '.':
                tensor[i, j, 2] = 1
            elif elemento == '@':
                tensor[i, j, 3] = 1
            elif elemento == '*':
                tensor[i, j, 4] = 1
            elif elemento == '+':
                tensor[i, j, 5] = 1
            elif elemento == ' ':
                tensor[i, j, 6] = 1
                
    return tensor


X = [tablero_a_tensor(tablero) for tablero in aug_boards]
print("Tensor")
with np.printoptions(threshold=np.inf):
    print(X[0])


max_height = max(tensor.shape[0] for tensor in X)
max_width = max(tensor.shape[1] for tensor in X)

X_padded = np.zeros((len(X), max_height, max_width, 7))

for idx, tensor in enumerate(X):
    height = tensor.shape[0]
    width = tensor.shape[1]
    X_padded[idx, :height, :width, :] = tensor

with np.printoptions(threshold=np.inf):
    print(X_padded[0])


'''print("Dimensiones de X_padded:", X_padded.shape)

# Verificar las dimensiones de X y y
#print("Dimensiones de X:", X.shape)
print("Dimensiones de y:", Y.shape)'''

Tensor
[[[0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 1.]
  [1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 1.]
  [1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 1.]
  [1. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 1.]
  [1. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 1.]
  [1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1.]
  [0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1.]
  [0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0.

'print("Dimensiones de X_padded:", X_padded.shape)\n\n# Verificar las dimensiones de X y y\n#print("Dimensiones de X:", X.shape)\nprint("Dimensiones de y:", Y.shape)'

FSN

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, GlobalAveragePooling2D, Dense, MaxPooling2D
from sklearn.model_selection import train_test_split


X_train, X_test, Y_train, Y_test = train_test_split(X_padded, Y, test_size=0.2, random_state=23)
print(len(X_train), len(Y_train), len(X_test), len(Y_test))


input_shape = (None, None, 7)  
inputs = Input(shape=input_shape)

# Arquitec
x = Conv2D(filters=32, kernel_size=(2, 2), activation='relu', padding='same')(inputs)
x = Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = GlobalAveragePooling2D()(x)
x = Dense(32, activation='relu')(x)
outputs = Dense(1, activation='sigmoid')(x)


model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(X_train, Y_train, epochs=300, batch_size=32, validation_data=(X_test, Y_test))


13627 13627 3407 3407


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None, None, 7)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, None, None, 32) │           928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, None, None, 32) │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, None, None, 64) │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 64)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,785 (120.25 KB)

 Trainable params: 30,785 (120.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/300
